In [2]:
from pprint import pprint
from paddlenlp import Taskflow

C:\Users\hyang\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\hyang\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\hyang\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
C:\Users\hyang\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\dtypes.py:585: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.object,
C:\Users\hyang\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\dtypes.py:627: DeprecationWarning: `np.object` is a deprecated alias

In [3]:
import pandas as pd
import numpy as np

In [4]:
import paddle

### 1.数据读取

In [47]:
df=pd.read_excel("fundManagers.xlsx")

In [48]:
df.head(10)

,证券代码,证券简称,简历
0,P0008YJBMC.WI,卞亚军基金经理业绩总指数,中国社会科学院研究生院金融学硕士。2004年7月-2006年7月就职于红塔证券资产管理部，任...
1,P0013JCJMC.WI,江俊晨基金经理业绩总指数,曾任复华投信资产管理(香港)有限公司投资研究部分析员，东吴基金管理有限公司投资研究部助理行业...
2,P0014FQSMC.WI,申峰旗基金经理业绩总指数,普林斯顿大学金融学硕士，15年证券从业经验。历任长城人寿投资总监、长城财富保险资产管理公司副...
3,P0014LYSMC.WI,孙连玉基金经理业绩总指数,硕士研究生，5年证券基金投资研究经验。2015年7月至2016年12月就职于中国中投证券股份...
4,P0015JFXMC.WI,冯佳基金经理业绩总指数,金融投资硕士。2005年5月至2009年5月任安永华明会计师事务所审计部高级审计师。2009...
5,P0016FGXMC.WI,葛飞基金经理业绩总指数,北京大学经济学硕士。曾任海航资本控股有限公司研究员、中诚信国际信用评级公司高级分析师、高级项...
6,P0016SYXMC.WI,袁素基金经理业绩总指数,北京大学西方经济学硕士。2011年至2020年，曾先后任安信证券固定收益部研究员、投资助理；...
7,P0016TQYMC.WI,杨添琦基金经理业绩总指数,2015年9月加入易方达基金管理有限公司，曾任行业研究员、投资经理；2016年6月加入易方达...
8,P0016WBHMC.WI,胡文伯基金经理业绩总指数,2014年4月加入易方达基金管理有限公司，曾任固定收益研究员，现任基金经理助理。2020年1...
9,P0016YHWMC.WI,武幼辉基金经理业绩总指数,曾任广发证券股份有限公司发展研究中心宏观策略分析师，融通基金管理有限公司研究部宏观策略研究员...


In [115]:
df.dropna(inplace=True)

### 2.使用飞浆微调后的模型进行简历提取

In [117]:
schema = ['公司', '部门', '职位', '大学','专业','学历','基金名称']
# 设定抽取目标和定制化模型权重路径
my_ie = Taskflow("information_extraction", schema=schema, task_path='./checkpoint/model_best')

[2022-07-30 12:36:17,961] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load './checkpoint/model_best'.


In [157]:
df_info=pd.DataFrame(columns=['公司','部门', '职位', '大学', '专业', '学历'])
for j in df.index:
    data=my_ie(df['简历'][j])[0]
    for k in ['公司','部门','职位','大学','学历','专业']:
        if k in data.keys():
            #用于记录该基金经理的公司、部门、职位信息，列表形式
            l=[]
            for i in range (0,len(data[k])):
                #print(data[k][i])
                item=data[k][i]
                if(item['probability']>0.95):
                    #print(item['text'])
                    if(item['text'] not in l):
                        l.append(item['text'])
                if(len(l)>0):
                    df_info.at[j,k]=np.array(l)
                #else:
                #    df_info.loc[j,k]=""

In [158]:
df_info

,公司,部门,职位,大学,专业,学历
0,"[华泰柏瑞基金管理有限公司, 摩根士丹利华鑫基金管理有限公司]",NaN,"[投资经理助理, 基金经理助理, 基金经理, 高级研究员]",NaN,[金融学],[硕士]
1,"[恒生前海基金管理有限公司, 复华投信资产管理(香港)有限公司, 东吴基金管理有限公司]","[投资研究部, 研究部]","[总经理, 助理行业研究员, 分析员, 基金经理]",NaN,NaN,NaN
2,"[中信保诚人寿资产管理中心, 嘉实基金, 新华基金管理股份有限公司, 长城财富保险资产管理公司]",NaN,"[副总监, 投资主管, 副总经理, 投资总监, 投资经理]",[普林斯顿大学],[金融学],[硕士]
3,[中国中投证券股份有限公司],NaN,[基金经理],NaN,NaN,[硕士研究生]
4,"[创金合信基金管理有限公司, 招商银行股份有限公司, 大成基金管理有限公司]","[固定收益部, 私人银行部, 固定收益总部, 资产管理部, 审计部]","[投研岗, 投资主办, 研究员, 高级审计师]",NaN,[金融投资],[硕士]
...,...,...,...,...,...,...
2951,"[南方基金, 南方成份]",NaN,"[基金经理, 高级研究员, 经理助理]",[密歇根大学],[经济学],[硕士学位]
2952,[上海东方证券资产管理有限公司],"[专户投资部, 研究部, 私募权益投资部, 公募集合权益投资部]","[总经理助理, 研究员, 基金经理, 高级研究员]",[上海交通大学],[管理学],[硕士]
2953,"[国联安基金管理有限公司, 上海雀沃信息技术有限公司]",NaN,"[资深分析师, 研究员]",NaN,NaN,NaN
2954,"[百度国际科技(深圳)有限公司, 上海桐昇通惠资产管理有限公司]",[智能投资部],"[基金经理, 量化研究员, 技术负责人]",[香港科技大学],NaN,[博士]


In [159]:
pd.concat([df,df_info],axis=1)

,证券代码,证券简称,简历,公司,部门,职位,大学,专业,学历
0,P0008YJBMC.WI,卞亚军基金经理业绩总指数,中国社会科学院研究生院金融学硕士。2004年7月-2006年7月就职于红塔证券资产管理部，任...,"[华泰柏瑞基金管理有限公司, 摩根士丹利华鑫基金管理有限公司]",NaN,"[投资经理助理, 基金经理助理, 基金经理, 高级研究员]",NaN,[金融学],[硕士]
1,P0013JCJMC.WI,江俊晨基金经理业绩总指数,曾任复华投信资产管理(香港)有限公司投资研究部分析员，东吴基金管理有限公司投资研究部助理行业...,"[恒生前海基金管理有限公司, 复华投信资产管理(香港)有限公司, 东吴基金管理有限公司]","[投资研究部, 研究部]","[总经理, 助理行业研究员, 分析员, 基金经理]",NaN,NaN,NaN
2,P0014FQSMC.WI,申峰旗基金经理业绩总指数,普林斯顿大学金融学硕士，15年证券从业经验。历任长城人寿投资总监、长城财富保险资产管理公司副...,"[中信保诚人寿资产管理中心, 嘉实基金, 新华基金管理股份有限公司, 长城财富保险资产管理公司]",NaN,"[副总监, 投资主管, 副总经理, 投资总监, 投资经理]",[普林斯顿大学],[金融学],[硕士]
3,P0014LYSMC.WI,孙连玉基金经理业绩总指数,硕士研究生，5年证券基金投资研究经验。2015年7月至2016年12月就职于中国中投证券股份...,[中国中投证券股份有限公司],NaN,[基金经理],NaN,NaN,[硕士研究生]
4,P0015JFXMC.WI,冯佳基金经理业绩总指数,金融投资硕士。2005年5月至2009年5月任安永华明会计师事务所审计部高级审计师。2009...,"[创金合信基金管理有限公司, 招商银行股份有限公司, 大成基金管理有限公司]","[固定收益部, 私人银行部, 固定收益总部, 资产管理部, 审计部]","[投研岗, 投资主办, 研究员, 高级审计师]",NaN,[金融投资],[硕士]
...,...,...,...,...,...,...,...,...,...
2951,P9925YZXMC.WI,张原基金经理业绩总指数,美国密歇根大学经济学硕士学位，具有基金从业资格。2006年4月加入南方基金，曾担任南方基金研...,"[南方基金, 南方成份]",NaN,"[基金经理, 高级研究员, 经理助理]",[密歇根大学],[经济学],[硕士学位]
2952,P9926SJQMC.WI,钱思佳基金经理业绩总指数,钱思佳女士，生于1983年，上海交通大学管理学硕士，自2008年起开始从事证券行业工作。历任...,[上海东方证券资产管理有限公司],"[专户投资部, 研究部, 私募权益投资部, 公募集合权益投资部]","[总经理助理, 研究员, 基金经理, 高级研究员]",[上海交通大学],[管理学],[硕士]
2953,P9927SGXMC.WI,高诗基金经理业绩总指数,2011年4月至2013年1月任国金证券研究所行业研究员；2013年2月至2014年3月任宏...,"[国联安基金管理有限公司, 上海雀沃信息技术有限公司]",NaN,"[资深分析师, 研究员]",NaN,NaN,NaN
2954,P9927WXXMC.WI,向伟基金经理业绩总指数,香港科技大学计算机科学与工程学系博士，曾任百度国际科技(深圳)有限公司技术负责人，上海桐昇通...,"[百度国际科技(深圳)有限公司, 上海桐昇通惠资产管理有限公司]",[智能投资部],"[基金经理, 量化研究员, 技术负责人]",[香港科技大学],NaN,[博士]


#### 2.1统计基金经理毕业院校

In [180]:
#统计毕业学校
data=df_info['大学'].dropna()
university_dict={}
for i in data.index:
    for j in range(0,len(data[i])):
        #如果这所院校出现过，那么更新key值
        if data[i][j] in university_dict.keys():
            v=university_dict[data[i][j]]
            university_dict.update({ data[i][j]:v+1})
        else:
            v=1
            university_dict.update({ data[i][j]:v})
university_dict=sorted(university_dict.items(),key = lambda x:x[1],reverse = True)

In [161]:
university_dict

[('北京大学', 129),
 ('复旦大学', 118),
 ('清华大学', 111),
 ('上海交通大学', 71),
 ('上海财经大学', 62),
 ('中国人民大学', 53),
 ('南开大学', 35),
 ('厦门大学', 30),
 ('中央财经大学', 27),
 ('武汉大学', 22),
 ('南京大学', 20),
 ('浙江大学', 17),
 ('中山大学', 16),
 ('中国科学院', 15),
 ('同济大学', 13),
 ('华中科技大学', 12),
 ('香港大学', 12),
 ('西南财经大学', 11),
 ('哥伦比亚大学', 11),
 ('香港中文大学', 10),
 ('华东师范大学', 10),
 ('中国科学技术大学', 8),
 ('哈尔滨工业大学', 8),
 ('四川大学', 7),
 ('中南财经政法大学', 7),
 ('北京交通大学', 6),
 ('华南理工大学', 6),
 ('天津大学', 6),
 ('香港科技大学', 6),
 ('吉林大学', 5),
 ('康奈尔大学', 5),
 ('芝加哥大学', 5),
 ('纽约大学', 5),
 ('南加州大学', 5),
 ('多伦多大学', 5),
 ('密歇根大学', 5),
 ('华东理工大学', 4),
 ('北京航空航天大学', 4),
 ('伯明翰大学', 4),
 ('北京邮电大学', 4),
 ('中国人民银行', 4),
 ('华威大学', 3),
 ('伊利诺伊大学', 3),
 ('西安交通大学', 3),
 ('加州大学', 3),
 ('东北财经大学', 3),
 ('牛津大学', 3),
 ('东南大学', 3),
 ('约克大学', 3),
 ('香港理工大学', 3),
 ('暨南大学', 3),
 ('悉尼大学', 3),
 ('上海大学', 3),
 ('中国科技大学', 3),
 ('北京理工大学', 3),
 ('普林斯顿大学', 2),
 ('杜克大学', 2),
 ('耶鲁大学', 2),
 ('布兰迪斯大学', 2),
 ('中南大学', 2),
 ('重庆大学', 2),
 ('北京师范大学', 2),
 ('伦敦政治经济学院', 2),
 ('麻省理工学院', 2),
 ('山

#### 2.2统计基金经理的学历

In [168]:
#统计文凭
data=df_info['学历'].dropna()
deploma_dict={}
for i in data.index:
    for j in range(0,len(data[i])):
        #如果这所院校出现过，那么更新key值
        if data[i][j] in deploma_dict.keys():
            v=deploma_dict[data[i][j]]
            deploma_dict.update({ data[i][j]:v+1})
        else:
            v=1
            deploma_dict.update({ data[i][j]:v})
deploma_dict=sorted(deploma_dict.items(),key = lambda x:x[1],reverse = True)

In [169]:
deploma_dict

[('硕士', 1206),
 ('博士', 179),
 ('硕士研究生', 152),
 ('学士', 127),
 ('硕士学位', 37),
 ('研究生', 20),
 ('博士研究生', 14),
 ('MBA', 11),
 ('本科', 5),
 ('博士学位', 4),
 ('学士学位', 2),
 ('本硕', 1)]

#### 2.3统计基金经理所学专业

In [175]:
#统计专业
data=df_info['专业'].dropna()
major_dict={}
for i in data.index:
    for j in range(0,len(data[i])):
        #如果这所院校出现过，那么更新key值
        if data[i][j] in major_dict.keys():
            v=major_dict[data[i][j]]
            major_dict.update({ data[i][j]:v+1})
        else:
            v=1
            major_dict.update({ data[i][j]:v})
major_dict=sorted(major_dict.items(),key = lambda x:x[1],reverse = True)

In [176]:
major_dict

[('经济学', 314),
 ('金融学', 267),
 ('管理学', 104),
 ('工商管理', 87),
 ('理学', 80),
 ('工学', 75),
 ('金融工程', 27),
 ('统计学', 16),
 ('会计学', 16),
 ('金融', 16),
 ('工程', 14),
 ('数量经济学', 12),
 ('金融数学', 10),
 ('金融专业', 8),
 ('数学', 8),
 ('工商管理学', 8),
 ('西方经济学', 7),
 ('应用经济学', 7),
 ('管理科学与工程', 7),
 ('应用数学', 7),
 ('产业经济学', 7),
 ('法学', 6),
 ('国民经济学', 6),
 ('企业管理', 6),
 ('世界经济', 5),
 ('金融工程专业', 5),
 ('计算机', 4),
 ('数学系', 4),
 ('管理科学与工程专业', 4),
 ('财务管理', 4),
 ('世界经济学', 4),
 ('数理金融', 4),
 ('精算学', 3),
 ('商学', 3),
 ('国际贸易学', 3),
 ('应用统计学', 3),
 ('哲学', 3),
 ('政治经济学', 3),
 ('国际金融学', 3),
 ('投资学', 3),
 ('计量金融学', 3),
 ('电子工程', 3),
 ('物理学', 3),
 ('计算机科学', 3),
 ('工程学', 3),
 ('国际经济与贸易', 3),
 ('计算机科学与技术', 3),
 ('细胞生物学', 2),
 ('数量金融学', 2),
 ('电子工程专业', 2),
 ('企业管理专业', 2),
 ('国际经济与金融学', 2),
 ('微电子学', 2),
 ('财政学', 2),
 ('经济专业', 2),
 ('化学', 2),
 ('技术经济及管理', 2),
 ('生物学', 2),
 ('国际商务', 2),
 ('生物医学', 2),
 ('电子信息科学与技术', 2),
 ('金融财务工商管理', 2),
 ('通信与信息系统', 2),
 ('金融工程学', 2),
 ('工科', 2),
 ('数学专业', 2),
 ('投资经济专业', 2),
 ('金融经济学', 2),
 ('管理科

#### 2.4统计基金经理就职公司（含曾供职公司和现供职公司）

In [200]:
#统计就职过的公司
data=df_info['公司'].dropna()
company_dict={}
for i in data.index:
    #由于某位基金经理简历中可能存在多次同一公司、部门名称，我们将这些内容去重后进行统计
    k={}
    k=k.fromkeys(data[i])
    for key in k.keys():
        if key in company_dict.keys():
            v=company_dict[key]
            company_dict.update({ key:v+1})
        else:
            v=1
            company_dict.update({ key:v})
company_dict=sorted(company_dict.items(),key = lambda x:x[1],reverse = True)

In [201]:
company_dict

[('华夏基金管理有限公司', 102),
 ('嘉实基金管理有限公司', 99),
 ('博时基金管理有限公司', 88),
 ('富国基金管理有限公司', 86),
 ('易方达基金管理有限公司', 83),
 ('鹏华基金管理有限公司', 81),
 ('广发基金管理有限公司', 79),
 ('招商基金管理有限公司', 76),
 ('南方基金', 66),
 ('大成基金管理有限公司', 64),
 ('中银基金管理有限公司', 62),
 ('国泰基金管理有限公司', 60),
 ('中欧基金管理有限公司', 55),
 ('交银施罗德基金管理有限公司', 51),
 ('长盛基金管理有限公司', 47),
 ('华泰柏瑞基金管理有限公司', 46),
 ('银华基金管理有限公司', 46),
 ('工银瑞信', 45),
 ('华安基金管理有限公司', 44),
 ('泰达宏利基金管理有限公司', 44),
 ('万家基金管理有限公司', 43),
 ('上投摩根基金管理有限公司', 43),
 ('前海开源基金管理有限公司', 42),
 ('天弘基金管理有限公司', 41),
 ('中国国际金融有限公司', 40),
 ('创金合信基金管理有限公司', 39),
 ('融通基金管理有限公司', 38),
 ('国泰君安证券股份有限公司', 38),
 ('安信基金管理有限责任公司', 38),
 ('国联安基金管理有限公司', 38),
 ('光大保德信基金管理有限公司', 38),
 ('国投瑞银基金管理有限公司', 37),
 ('工银瑞信基金管理有限公司', 36),
 ('海富通基金管理有限公司', 36),
 ('中信证券股份有限公司', 35),
 ('宝盈基金管理有限公司', 35),
 ('汇添富基金管理股份有限公司', 35),
 ('华宝兴业基金管理有限公司', 35),
 ('长城基金管理有限公司', 34),
 ('长信基金管理有限责任公司', 33),
 ('汇丰晋信基金管理有限公司', 33),
 ('农银汇理基金管理有限公司', 32),
 ('平安资产管理有限责任公司', 31),
 ('诺安基金管理有限公司', 31),
 ('海通证券股份有限公司', 30),
 ('兴业基金管理有限公司', 30),
 ('兴业

#### 2.5统计基金经理供职部门

In [206]:
#统计就职过的部门
data=df_info['部门'].dropna()
department_dict={}
for i in data.index:
    #由于某位基金经理简历中可能存在多次同一公司、部门名称，我们将这些内容去重后进行统计
    k={}
    k=k.fromkeys(data[i])
    for key in k.keys():
        if key in department_dict.keys():
            v=department_dict[key]
            department_dict.update({ key:v+1})
        else:
            v=1
            department_dict.update({ key:v})
department_dict=sorted(department_dict.items(),key = lambda x:x[1],reverse = True)

In [207]:
department_dict

[('研究部', 432),
 ('固定收益部', 346),
 ('权益投资部', 131),
 ('资产管理部', 126),
 ('投资研究部', 113),
 ('研究发展部', 82),
 ('投资部', 80),
 ('固定收益投资部', 76),
 ('量化投资部', 74),
 ('金融市场部', 70),
 ('专户投资部', 68),
 ('投资管理部', 64),
 ('股票投资部', 59),
 ('基金投资部', 55),
 ('国际业务部', 51),
 ('金融工程部', 36),
 ('指数投资部', 36),
 ('交易部', 35),
 ('权益研究部', 28),
 ('基金管理部', 27),
 ('资产配置部', 25),
 ('投资银行部', 24),
 ('指数与量化投资部', 22),
 ('固定收益总部', 21),
 ('证券投资部', 21),
 ('交易管理部', 20),
 ('数量投资部', 19),
 ('资产管理总部', 19),
 ('固定收益研究部', 17),
 ('机构投资部', 17),
 ('风险管理部', 17),
 ('国际投资部', 17),
 ('投资管理一部', 15),
 ('固收投资部', 15),
 ('集中交易部', 15),
 ('权益投资二部', 15),
 ('研究所', 14),
 ('市场部', 13),
 ('专户理财部', 12),
 ('数量化投资部', 12),
 ('基金业务部', 11),
 ('资金部', 11),
 ('权益投资本部', 11),
 ('债券投资部', 11),
 ('公募权益投资部', 11),
 ('资产管理业务总部', 10),
 ('产品开发部', 10),
 ('财富管理部', 9),
 ('公募投资部', 9),
 ('现金管理部', 9),
 ('研究策划部', 9),
 ('绝对收益投资部', 8),
 ('公募事业部', 8),
 ('组合管理部', 8),
 ('集中交易室', 8),
 ('研究生部', 8),
 ('审计部', 7),
 ('投资管理三部', 7),
 ('信用评估部', 7),
 ('投资管理总部', 7),
 ('信息技术部', 7),
 ('资金交易部', 7),
 ('中央交易室', 

#### 2.6统计基金经理担任职务

In [211]:
#统计担任过的职务
data=df_info['职位'].dropna()
job_dict={}
for i in data.index:
    #由于某位基金经理简历中可能存在多次同一公司、部门名称，我们将这些内容去重后进行统计
    k={}
    k=k.fromkeys(data[i])
    for key in k.keys():
        if key in job_dict.keys():
            v=job_dict[key]
            job_dict.update({ key:v+1})
        else:
            v=1
            job_dict.update({ key:v})
job_dict=sorted(job_dict.items(),key = lambda x:x[1],reverse = True)

In [212]:
job_dict

[('基金经理', 1779),
 ('研究员', 1212),
 ('投资经理', 675),
 ('基金经理助理', 618),
 ('高级研究员', 274),
 ('分析师', 181),
 ('副总经理', 180),
 ('总经理', 149),
 ('交易员', 145),
 ('副总监', 142),
 ('投资经理助理', 117),
 ('总经理助理', 116),
 ('总监', 100),
 ('高级分析师', 95),
 ('助理研究员', 88),
 ('高级经理', 71),
 ('投资总监', 67),
 ('经理助理', 60),
 ('高级投资经理', 56),
 ('经理', 52),
 ('负责人', 50),
 ('总监助理', 49),
 ('行业研究员', 48),
 ('研究总监', 43),
 ('项目经理', 38),
 ('产品经理', 36),
 ('副总裁', 32),
 ('委员', 29),
 ('投资副总监', 29),
 ('投资助理', 27),
 ('量化研究员', 27),
 ('资深研究员', 27),
 ('信用研究员', 23),
 ('首席分析师', 23),
 ('专户投资经理', 22),
 ('高级交易员', 22),
 ('债券交易员', 22),
 ('投资主办', 20),
 ('董事总经理', 20),
 ('投资主办人', 20),
 ('助理分析师', 19),
 ('策略分析师', 19),
 ('研究主管', 18),
 ('债券研究员', 17),
 ('高级副总裁', 17),
 ('执行董事', 17),
 ('审计师', 16),
 ('工程师', 16),
 ('研究助理', 16),
 ('副经理', 15),
 ('首席策略分析师', 15),
 ('组长', 15),
 ('高级债券研究员', 15),
 ('行业分析师', 15),
 ('发起式联接基金基金经理', 15),
 ('交易主管', 14),
 ('策略研究员', 14),
 ('董事', 14),
 ('分析员', 13),
 ('高级产品经理', 13),
 ('投资顾问', 13),
 ('量化分析师', 13),
 ('高级行业研究员', 13),
 ('职员', 13),
 

In [204]:
#统计就职过的公司
data=df_info['部门'].dropna()
department_dict={}
for i in data.index:
    #由于某位基金经理简历中可能存在多次同一公司、部门名称，我们将这些内容去重后进行统计
    k={}
    k=k.fromkeys(data[i])
    for key in k.keys():
        if key in department_dict.keys():
            v=department_dict[key]
            department_dict.update({ key:v+1})
        else:
            v=1
            department_dict.update({ key:v})
department_dict=sorted(department_dict.items(),key = lambda x:x[1],reverse = True)

In [205]:
department_dict

[('研究部', 432),
 ('固定收益部', 346),
 ('权益投资部', 131),
 ('资产管理部', 126),
 ('投资研究部', 113),
 ('研究发展部', 82),
 ('投资部', 80),
 ('固定收益投资部', 76),
 ('量化投资部', 74),
 ('金融市场部', 70),
 ('专户投资部', 68),
 ('投资管理部', 64),
 ('股票投资部', 59),
 ('基金投资部', 55),
 ('国际业务部', 51),
 ('金融工程部', 36),
 ('指数投资部', 36),
 ('交易部', 35),
 ('权益研究部', 28),
 ('基金管理部', 27),
 ('资产配置部', 25),
 ('投资银行部', 24),
 ('指数与量化投资部', 22),
 ('固定收益总部', 21),
 ('证券投资部', 21),
 ('交易管理部', 20),
 ('数量投资部', 19),
 ('资产管理总部', 19),
 ('固定收益研究部', 17),
 ('机构投资部', 17),
 ('风险管理部', 17),
 ('国际投资部', 17),
 ('投资管理一部', 15),
 ('固收投资部', 15),
 ('集中交易部', 15),
 ('权益投资二部', 15),
 ('研究所', 14),
 ('市场部', 13),
 ('专户理财部', 12),
 ('数量化投资部', 12),
 ('基金业务部', 11),
 ('资金部', 11),
 ('权益投资本部', 11),
 ('债券投资部', 11),
 ('公募权益投资部', 11),
 ('资产管理业务总部', 10),
 ('产品开发部', 10),
 ('财富管理部', 9),
 ('公募投资部', 9),
 ('现金管理部', 9),
 ('研究策划部', 9),
 ('绝对收益投资部', 8),
 ('公募事业部', 8),
 ('组合管理部', 8),
 ('集中交易室', 8),
 ('研究生部', 8),
 ('审计部', 7),
 ('投资管理三部', 7),
 ('信用评估部', 7),
 ('投资管理总部', 7),
 ('信息技术部', 7),
 ('资金交易部', 7),
 ('中央交易室', 